## Loading Libraries

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import xgboost as xgb
import lightgbm as lgb
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import make_scorer

## Loading Data
- train.csv - historical data including Sales
- test.csv - historical data excluding Sales
- store.csv - supplemental information about the stores

In [2]:
train_df = pd.read_csv('../input/train.csv')
store_df = pd.read_csv('../input/store.csv')
test_df = pd.read_csv('../input/test.csv')

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## Data Assess
- Id - an Id that represents a (Store, Date) duple within the test set
- Store - a unique Id for each store
- Sales - the turnover for any given day (this is what you are predicting)
- Customers - the number of customers on a given day
- Open - an indicator for whether the store was open: 0 = closed, 1 = open
- StateHoliday - indicates a state holiday. Normally all stores, with few exceptions, are closed on state holidays. Note that all schools are  closed on public holidays and weekends. a = public holiday, b = Easter holiday, c = Christmas, 0 = None
- SchoolHoliday - indicates if the (Store, Date) was affected by the closure of public schools
- StoreType - differentiates between 4 different store models: a, b, c, d
- Assortment - describes an assortment level: a = basic, b = extra, c = extended
- CompetitionDistance - distance in meters to the nearest competitor store
- CompetitionOpenSince[Month/Year] - gives the approximate year and month of the time the nearest competitor was opened
- Promo - indicates whether a store is running a promo on that day
- Promo2 - Promo2 is a continuing and consecutive promotion for some stores: 0 = store is not participating, 1 = store is participating
- Promo2Since[Year/Week] - describes the year and calendar week when the store started participating in Promo2
- PromoInterval - describes the consecutive intervals Promo2 is started, naming the months the promotion is started anew. E.g.    "Feb,May,Aug,Nov" means each round starts in February, May, August, November of any given year for that store

#### train data

#### store data

#### test data

## Data Wrangle

### Tidy
- merge the train_df and store_df
- merge the test_df and store_df

#### merge the train_df and store_df
using pandas.merge function to merge train_df, test_df and store_df.

#### Code

In [3]:
train_store_df = pd.merge(train_df, store_df, on='Store')
test_store_df = pd.merge(test_df, store_df, on='Store')

#### Test

### Quality

`train_store_df`:
- incorrect data type: `Date`
- missing data : `CompetitionDistance`, `CompetitionOpenSinceMonth`, `CompetitionOpenSinceYear`, `Promo2SinceWeek`, `Promo2SinceYear`, `PromoInterval`

`test_store_df`:
- incorrect data type: `Date`
- missing data : `Open`, `CompetitionDistance`, `CompetitionOpenSinceMonth`, `CompetitionOpenSinceYear`, `Promo2SinceWeek`, `Promo2SinceYear`, `PromoInterval`

### `train_store_df` : 

#### incorrect data type: `Date`
Using pandas.to_datetime to do it.

#### code

In [4]:
train_store_df.Date = pd.to_datetime(train_store_df.Date)

#### Test

#### missing data : `CompetitionDistance`, `CompetitionOpenSinceMonth`, `CompetitionOpenSinceYear`, `Promo2SinceWeek`, `Promo2SinceYear`, `PromoInterval`

Using pandas.DataFrame.fillna to fill `CompetitionDistance`,`CompetitionOpenSinceMonth` and `CompetitionOpenSinceYear`with 0.

Using pandas.DataFrame.fillna to fill `Promo2SinceWeek`,`Promo2SinceYear` and `PromoInterval` with 0.


#### Code

In [5]:
train_store_df.CompetitionDistance.fillna(0, inplace=True)
train_store_df.CompetitionOpenSinceMonth.fillna(0, inplace=True)
train_store_df.CompetitionOpenSinceYear.fillna(0, inplace=True)

In [6]:
train_store_df.Promo2SinceWeek.fillna(0, inplace=True)
train_store_df.Promo2SinceYear.fillna(0, inplace=True)
train_store_df.PromoInterval.fillna(0, inplace=True)

#### Test

### `test_store_df`:

#### incorrect data type: `Date`
Using pandas.to_datetime to do it.

#### Code

In [7]:
test_store_df.Date = pd.to_datetime(test_store_df.Date)

#### Test

#### missing data : `Open`
Using pandas.DataFrame.fillna to fill Open with 1.

#### Code

In [8]:
test_store_df.Open.fillna(1, inplace=True)

In [9]:
#test
test_store_df.CompetitionDistance.fillna(0, inplace=True)
test_store_df.CompetitionOpenSinceMonth.fillna(0, inplace=True)
test_store_df.CompetitionOpenSinceYear.fillna(0, inplace=True)

test_store_df.Promo2SinceWeek.fillna(0, inplace=True)
test_store_df.Promo2SinceYear.fillna(0, inplace=True)
test_store_df.PromoInterval.fillna(0, inplace=True)

#### Test

## EDA

### Univariate Analysis

#### Sales

Sales is positive skewed distribution。

#### Date

The Date in train data is from 2013-01-01 to 2015-07-31, while the date in test data is from 2015-08-01 to 2015-09-17.

#### Customers

Customers has a lot of zero values and is positive skewed distribution.

#### CompetitionDistance

CompetitionDistance is positive skewed distribution.

#### CompetitionOpenSinceMonth

In [10]:
train_store_df.CompetitionOpenSinceMonth.value_counts()

0.0     323348
9.0     114254
4.0      87076
11.0     84455
3.0      63548
7.0      59434
12.0     57896
10.0     55622
6.0      45444
5.0      39608
2.0      37886
8.0      36186
1.0      12452
Name: CompetitionOpenSinceMonth, dtype: int64

The zero value accounts for the majority.

#### CompetitionOpenSinceYear

The zero value accounts for the majority.

#### Promo2SinceWeek

The zero value accounts for the majority.

#### Promo2SinceYear

In [11]:
train_store_df.Promo2SinceYear.value_counts()

0.0       508031
2011.0    115056
2013.0    110464
2014.0     79922
2012.0     73174
2009.0     65270
2010.0     56240
2015.0      9052
Name: Promo2SinceYear, dtype: int64

The zero value accounts for the majority.

#### PromoInterval

The zero value accounts for the majority. Jan,Apr,Jul,Oct follows the following.

### Construct some new variables

#### Construct new variables `Year`, `Month`, `Day` and `WeekOfYear`  based on `Date` .

In [12]:
#train_store_df
train_store_df['Year'] = train_store_df.Date.dt.year
train_store_df['Month'] = train_store_df.Date.dt.month
train_store_df['Day'] = train_store_df.Date.dt.day
train_store_df['WeekOfYear'] = train_store_df.Date.dt.weekofyear

In [13]:
#test_store_df
test_store_df['Year'] = test_store_df.Date.dt.year
test_store_df['Month'] = test_store_df.Date.dt.month
test_store_df['Day'] = test_store_df.Date.dt.day
test_store_df['WeekOfYear'] = test_store_df.Date.dt.weekofyear

CompetionOpen en PromoOpen from https://www.kaggle.com/ananya77041/rossmann-store-sales/randomforestpython/code

#### Construct new variables `CompetitionOpen`

In [14]:
#train_store_df
train_store_df['CompetitionOpen'] = 12*(train_store_df.Year - train_store_df.CompetitionOpenSinceYear) + \
                                   (train_store_df.Month - train_store_df.CompetitionOpenSinceMonth)
#test_store_df
test_store_df['CompetitionOpen'] = 12*(test_store_df.Year - test_store_df.CompetitionOpenSinceYear) + \
                                   (test_store_df.Month - test_store_df.CompetitionOpenSinceMonth)

#### Construct new variables `PromoOpen` 

In [15]:
#train_store_df
train_store_df['PromoOpen'] = 12*(train_store_df.Year - train_store_df.Promo2SinceYear) + \
                                 (train_store_df.WeekOfYear - train_store_df.Promo2SinceWeek) / 4.0

train_store_df['PromoOpen'] = train_store_df.PromoOpen.apply(lambda x: x if x > 0 else 0)
train_store_df.loc[train_store_df.Promo2SinceYear == 0, 'PromoOpen'] = 0

#test_store_df
test_store_df['PromoOpen'] = 12*(test_store_df.Year - test_store_df.Promo2SinceYear) + \
                                (test_store_df.WeekOfYear - test_store_df.Promo2SinceWeek) / 4.0

test_store_df['PromoOpen'] = test_store_df.PromoOpen.apply(lambda x: x if x > 0 else 0)
test_store_df.loc[test_store_df.Promo2SinceYear == 0, 'PromoOpen'] = 0

### Bivariate Analysis

#### Sales & Open

There are some opened stores which have zero sales.

#### Analysis of sales!=0 and open != 0

In [16]:
train_store_sales_df = train_store_df[(train_store_df.Sales != 0) & (train_store_df.Open != 0)]

####  Sales & DayOfWeek

Sales on Saturday are the lowest.

There seems to be some abnormal points. The daily sales are almost the same.

####  Sales & WeekOfYear

#### Sales & Year

The three years sales are almost the same.

#### Sales & Month

The December sales is the largest.

#### Sales & Day

we see that sales are the most at the beginning of the month, the monthly, and the end of the month.
There is a strong rule between the two variables.

#### Sales & Customers

There is a strong positive correlation between sales and customer.

#### Sales & Promo

We can know that promotion is benifit to improve sales.

#### Sales & StateHoliday

In [17]:
train_store_sales_df.StateHoliday = train_store_sales_df.StateHoliday.astype('str')
train_store_sales_df[train_store_sales_df.StateHoliday == '0'].StateHoliday = train_store_sales_df[train_store_sales_df.StateHoliday == '0'].StateHoliday.astype('int')

/opt/conda/lib/python3.6/site-packages/pandas/core/generic.py:3643: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


#### Sales & SchoolHoliday

#### Sales & StoreType

Store sales of 'b' type are the most.

#### Sales & Assortment

Store sales of  'b' assortment are the most.

#### Sales & CompetitionDistance

There is no correlation between sales and CompetitionDistance.

#### Sales & CompetitionOpen

There is a weak rule between sales and CompetitionOpen.

#### Sales & Promo2

The stores with promos tend to make lower sales. This does not necessary mean that the promos don’t help or are counterproductive. They are possibly measures that are taken mainly by stores with low sales in the first place.

#### Sales & PromoOpen

It seems that the longer the time is, the bigger the sales are.

There is no correlation between sales and Promo2SinceYear.

#### Sales & PromoInterval

There is no correlation between sales and PromoInterval.

#### Brief Summary

From above bivariate analysis, we can see that there is a strong positive correlation between sales and customer.
There is a certain relationship between variables Sales and  Day, DayofWeek, WeekofYear, CompetitionOpen, PromoOpen.

### Multivariate Analysis

#### Sales & DayOfWeek & StoreType & Promo 

We can see that stores are never promoted at the weekend.

#### Sales & WeekOfYear & StoreType & Promo 

#### Sales & Month & StoreType & Promo 

Sales promotion can improve sales to a certain extent. There is a distinct increase in sales in the fourth quarter.

#### Sales & Month & Assortment	 & StoreType

Only 'b' type stores have 'b' assortment level.

#### Sales & Day & StoreType & Promo 

All types of stores have a similar trend, and sales are the most at the beginning of the month, the monthly, and the end of the month.

#### Sales & Month & DayOfWeek & StoreType

We see that stores of StoreType C are all closed on Sundays, whereas others are most of the time opened. Interestingly enough, stores of StoreType D are closed on Sundays only from October to December.

#### Sales & StateHoliday & Promo & StoreType

Only B type stores will be promoted on Easter, and there is no promotion on Christmas.

#### Sales & SchoolHoliday & Promo & StoreType

## Feature Engineering

In [18]:
train_store_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1017209 entries, 0 to 1017208
Data columns (total 24 columns):
Store                        1017209 non-null int64
DayOfWeek                    1017209 non-null int64
Date                         1017209 non-null datetime64[ns]
Sales                        1017209 non-null int64
Customers                    1017209 non-null int64
Open                         1017209 non-null int64
Promo                        1017209 non-null int64
StateHoliday                 1017209 non-null object
SchoolHoliday                1017209 non-null int64
StoreType                    1017209 non-null object
Assortment                   1017209 non-null object
CompetitionDistance          1017209 non-null float64
CompetitionOpenSinceMonth    1017209 non-null float64
CompetitionOpenSinceYear     1017209 non-null float64
Promo2                       1017209 non-null int64
Promo2SinceWeek              1017209 non-null float64
Promo2SinceYear              101720

In [19]:
test_store_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41088 entries, 0 to 41087
Data columns (total 23 columns):
Id                           41088 non-null int64
Store                        41088 non-null int64
DayOfWeek                    41088 non-null int64
Date                         41088 non-null datetime64[ns]
Open                         41088 non-null float64
Promo                        41088 non-null int64
StateHoliday                 41088 non-null object
SchoolHoliday                41088 non-null int64
StoreType                    41088 non-null object
Assortment                   41088 non-null object
CompetitionDistance          41088 non-null float64
CompetitionOpenSinceMonth    41088 non-null float64
CompetitionOpenSinceYear     41088 non-null float64
Promo2                       41088 non-null int64
Promo2SinceWeek              41088 non-null float64
Promo2SinceYear              41088 non-null float64
PromoInterval                41088 non-null object
Year            

#### Encoding

In [20]:
#train
mappings = {'0':0,'a':1, 'b':2, 'c':3, 'd':4}
train_store_df.StateHoliday.replace(mappings, inplace=True)
train_store_df.StoreType.replace(mappings, inplace=True)
train_store_df.Assortment.replace(mappings, inplace=True)

#test
test_store_df.StateHoliday.replace(mappings, inplace=True)
test_store_df.StoreType.replace(mappings, inplace=True)
test_store_df.Assortment.replace(mappings, inplace=True)

#### Choice X,y for training

In [21]:
#Choice the Date within 2015/7/18-2015/7/31 for valid data.
Xy_valid_df = train_store_df[train_store_df.Date > '7/17/2015']
Xy_train_df = train_store_df[train_store_df.Date <= '7/17/2015']

# Consider only open stores which Sales bigger then zero for training.
train_drop_features = ['Customers', 'Sales','Date','CompetitionOpenSinceMonth','CompetitionOpenSinceYear',
                       'Promo2SinceWeek','Promo2SinceYear','PromoInterval']
#train data
X_train = Xy_train_df[(Xy_train_df.Open != 0 ) & (Xy_train_df.Sales != 0)].drop(train_drop_features, axis=1)
y_train = Xy_train_df[(Xy_train_df.Open != 0 ) & (Xy_train_df.Sales != 0)].Sales
y_train = np.log1p(y_train)

#valid data
X_valid = Xy_valid_df[(Xy_valid_df.Open != 0 ) & (Xy_valid_df.Sales != 0)].drop(train_drop_features, axis=1)
y_valid = Xy_valid_df[(Xy_valid_df.Open != 0 ) & (Xy_valid_df.Sales != 0)].Sales
y_valid = np.log1p(y_valid)

In [22]:
test_drop_features = ['Id','Date','CompetitionOpenSinceMonth','CompetitionOpenSinceYear', 
                      'Promo2SinceWeek','Promo2SinceYear', 'PromoInterval']
X_test = test_store_df.drop(test_drop_features, axis=1)

## Model

In [23]:
"""
def ToWeight(y):
    w = np.zeros(y.shape, dtype=float)
    ind = y != 0
    w[ind] = 1./(y[ind]**2)
    return w

def rmspe(yhat, y):
    w = ToWeight(y)
    rmspe = np.sqrt(np.mean( w * (y - yhat)**2 ))
    return rmspe


def rmspe_xg(yhat, y):
    # y = y.values
    y = y.get_label()
    y = np.exp(y) - 1
    yhat = np.exp(yhat) - 1
    w = ToWeight(y)
    rmspe = np.sqrt(np.mean(w * (y - yhat)**2))
    return "rmspe", rmspe
"""
def rmspe(y, yhat):
    return np.sqrt(np.mean((yhat/y-1) ** 2))

#xgboost
def rmspe_xg(yhat, y):
    y = np.expm1(y.get_label())
    yhat = np.expm1(yhat)
    return "rmspe", rmspe(y,yhat)

#lightgbm
def rmspe_lgb(yhat, y):
    y = np.expm1(y.get_label())
    yhat = np.expm1(yhat)
    return "rmspe", rmspe(y,yhat), False

#sklearn interface
def rmspe_gscv(y, yhat):
    y = np.expm1(y)
    yhat = np.expm1(yhat)
    return rmspe(y, yhat)

rmspe_score = make_scorer(rmspe_gscv)
kfold = KFold(n_splits=5, random_state=23)

#### LightGBM

In [24]:
dtrain = lgb.Dataset(X_train, label=y_train)
dvalid = lgb.Dataset(X_valid, label=y_valid)

####  Origin Model

lgbm_params = {'task': 'train',
               'boosting_type': 'gbdt',
               'objective': 'regression',
               'bagging_seed': 3,
               'feature_fraction_seed': 2,
              }

%%time
lgb_model = lgb.train(lgbm_params, dtrain, num_boost_round=10000, valid_sets=[dvalid,dtrain], valid_names=['eval', 'train'], 
                      early_stopping_rounds=50, feval=rmspe_lgb, verbose_eval=True)
print("Validating")
yhat = lgb_model.predict(X_valid)
error = rmspe(np.expm1(y_valid), np.expm1(yhat))
print('RMSPE: {:.6f}'.format(error))

%%time
lgb_test_prod = lgb_model.predict(X_test)
lgb_test_prod = np.expm1(lgb_test_prod)
sub_df = pd.DataFrame({"Id":test_store_df["Id"].values})
sub_df["Sales"] = lgb_test_prod
sub_df.to_csv("origin_lgb_submission.csv", index=False)

#### lmproment Model after Tuning

In [25]:
lgbm_params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'regression',
    "learning_rate": 1,
    "num_leaves": 140,
    "max_bin": 255,
    "min_data_in_leaf ": 20,
    'device': 'gpu',
    'gpu_platform_id': 0,
    'gpu_device_id': 0
    }
lgb_best_params = {}
lgb_best_params["num_leaves"] = 140

In [26]:
min_merror = np.inf
for max_bin in range(255,260,5):
    for min_data_in_leaf in range(20, 100, 10):
        lgbm_params["max_bin"] = max_bin
        lgbm_params["min_data_in_leaf"] = min_data_in_leaf
    
        cv_results = lgb.cv(params = lgbm_params,
                            train_set = dtrain,
                            num_boost_round=3000,
                            stratified = False,
                            nfold = 5,
                            feval = rmspe_lgb,
                            verbose_eval=50,
                            seed = 23,
                            early_stopping_rounds = 30)
        mean_error = min(cv_results['rmspe-mean'])
        boost_rounds = np.argmin(cv_results['rmspe-mean'])
    
        if mean_error < min_merror:
            min_merror = mean_error
            lgb_best_params["max_bin"] = max_bin
            lgb_best_params["min_data_in_leaf"] = min_data_in_leaf

lgbm_params["max_bin"] = lgb_best_params["max_bin"]
lgbm_params["min_data_in_leaf"] = lgb_best_params["min_data_in_leaf"]
lgb_best_params

LightGBMError: b'GPU Tree Learner was not enabled in this build. Recompile with CMake option -DUSE_GPU=1'

In [ ]:
min_merror = np.inf
for feature_fraction in [i/10.0 for i in range(3,11)]:
    for bagging_fraction in [i/10.0 for i in range(3,11)]:
        for bagging_freq in range(0,10,2):
            lgbm_params["feature_fraction"] = feature_fraction
            lgbm_params["bagging_fraction"] = bagging_fraction
            lgbm_params["bagging_freq"] = bagging_freq 
        
            cv_results = lgb.cv(params = lgbm_params,
                                train_set = dtrain,
                                num_boost_round = 3000,
                                stratified = False,
                                nfold = 5,
                                feval = rmspe_lgb,
                                verbose_eval = 100,
                                seed = 23,
                                early_stopping_rounds = 30)
            mean_error = min(cv_results['rmspe-mean'])
            boost_rounds = np.argmin(cv_results['rmspe-mean'])
    
            if mean_error < min_merror:
                min_merror = mean_error
                lgb_best_params["feature_fraction"] = feature_fraction
                lgb_best_params["bagging_fraction"] = bagging_fraction
                lgb_best_params["bagging_freq"] = bagging_freq

lgbm_params["feature_fraction"] = lgb_best_params["feature_fraction"]
lgbm_params["bagging_fraction"] = lgb_best_params["bagging_fraction"]
lgbm_params["bagging_freq"] = lgb_best_params["bagging_freq"]
lgb_best_params

In [ ]:
min_merror = np.inf
for lambda_l1 in [i/10.0 for i in range(0,11)]:
    for lambda_l2 in [i/10.0 for i in range(0,11)]:
        for min_split_gain in [i/10.0 for i in range(0,11)]:
            lgbm_params["lambda_l1"] = lambda_l1
            lgbm_params["lambda_l2"] = lambda_l2
            lgbm_params["min_split_gain"] = min_split_gain 
    
            cv_results = lgb.cv(params = lgbm_params,
                                train_set = dtrain,
                                num_boost_round = 3000,
                                stratified = False,
                                nfold = 5,
                                feval = rmspe_lgb,
                                verbose_eval = 100,
                                seed = 23,
                                early_stopping_rounds = 30)
            mean_error = min(cv_results['rmspe-mean'])
            boost_rounds = np.argmin(cv_results['rmspe-mean'])
    
            if mean_error < min_merror:
                min_merror = mean_error
                lgb_best_params["lambda_l1"] = lambda_l1
                lgb_best_params["lambda_l2"] = lambda_l2
                lgb_best_params["min_split_gain"] = min_split_gain

lgbm_params["lambda_l1"] = lgb_best_params["lambda_l1"]
lgbm_params["lambda_l2"] = lgb_best_params["lambda_l2"]
lgbm_params["min_split_gain"] = lgb_best_params["min_split_gain"]
lgb_best_params